In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
import pandas as pd
from PIL import Image

# Hybrid Model
Combining MRI Scan and Sequential Health Data Analysis

The following cells implement a hybrid model that combines predictions from:
1. A CNN model trained on MRI scans
2. An ensemble model trained on sequential health data (demographics, clinical scores, etc.)

This dual-stream approach enables more comprehensive Alzheimer's assessment by leveraging both imaging biomarkers and clinical indicators.


In [7]:
class HybridModel:
    def __init__(self, cnn_model_path, sequential_model_path):
        """
        Initialize the hybrid model with paths to both trained models
        """
        # Load the CNN model for MRI images
        self.cnn_model = load_model(cnn_model_path)
        
        # Load the sequential data model
        try:
            self.sequential_model = joblib.load(sequential_model_path)
        except Exception as e:
            raise ValueError(f"Unable to load sequential model - file may be corrupted or incompatible: {str(e)}")
    def preprocess_mri(self, image_path):
        """
        Preprocess MRI image for CNN model
        """
        # Load and preprocess image
        img = Image.open(image_path).convert('L')  # Convert to grayscale
        img = img.resize((128, 128))  # Resize to expected input size
        img_array = np.array(img)
        img_array = img_array / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array = np.expand_dims(img_array, axis=-1)  # Add channel dimension
        return img_array
    
    def predict(self, mri_image_path, sequential_data):
        """
        Make predictions using both models and combine results
        
        Parameters:
        mri_image_path (str): Path to MRI image
        sequential_data (pd.DataFrame): DataFrame containing sequential health data
        
        Returns:
        dict: Combined prediction results
        """
        # Get CNN prediction for MRI
        mri_input = self.preprocess_mri(mri_image_path)
        cnn_pred = self.cnn_model.predict(mri_input)
        cnn_class = np.argmax(cnn_pred, axis=1)[0]
        cnn_confidence = np.max(cnn_pred)
        
        # Get sequential model prediction
        seq_pred = self.sequential_model.predict_proba(sequential_data)
        seq_class = self.sequential_model.predict(sequential_data)[0]
        seq_confidence = np.max(seq_pred, axis=1)[0]
        
        # Combine predictions (weighted average)
        # You can adjust these weights based on model performance
        cnn_weight = 0.6
        seq_weight = 0.4
        
        combined_confidence = (cnn_confidence * cnn_weight + 
                             seq_confidence * seq_weight)
        
        # If predictions agree, use that class. If not, use the one with higher confidence
        if cnn_class == seq_class:
            final_class = cnn_class
        else:
            final_class = cnn_class if cnn_confidence > seq_confidence else seq_class
            
        return {
            'final_prediction': final_class,
            'combined_confidence': combined_confidence,
            'cnn_prediction': cnn_class,
            'cnn_confidence': cnn_confidence,
            'sequential_prediction': seq_class,
            'sequential_confidence': seq_confidence
        }

In [9]:
# Example usage
cnn_model_path = '/home/rohitb/projects/ml/Project/mri_data_analysis/trained_models/cnn_model_3.h5'
sequential_model_path = '/home/rohitb/projects/ml/Project/sequential_data_analysis/trained_models/ensamble_nb_xgb.joblib'

# Initialize hybrid model
hybrid_model = HybridModel(cnn_model_path, sequential_model_path)

# Note: To use this model, you'll need:
# 1. MRI image file path
# 2. DataFrame containing sequential health data with appropriate features


/home/rohitb/projects/ml/Project/venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rohitb/projects/ml/Project/venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Testing the Combined Model Performance

The hybrid model's performance will be evaluated using a comprehensive testing approach:

1. **Test Dataset Preparation**
   - Split OASIS dataset into training/testing sets
   - Ensure balanced representation of different CDR scores
   - Prepare both MRI scans and corresponding sequential data

2. **Performance Metrics**
   - Accuracy, Precision, Recall, F1-score
   - Confusion matrix for multi-class classification
   - ROC curves and AUC scores
   - Model confidence analysis

3. **Comparative Analysis**
   - Compare hybrid model performance vs individual models
   - Analyze cases where models disagree
   - Evaluate confidence weighting effectiveness

4. **Visualization**
   - Performance metrics visualization
   - Confidence distribution plots
   - Error analysis charts

Results will be documented and compared against baseline individual model performance to demonstrate the advantages of the hybrid approach.
